# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
cities_examined = pd.read_csv("../output_data/Retrieved_City_Weather_Data.csv")
cities_examined.head()

,Unnamed: 0,City,Approx. Latitude,Approx. Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Actual Latitude,Actual Longitude,Country,Datetime (Local)
0,0,umm lajj,23.342676,36.240456,81.81,60,0,1.28,25.02,37.27,SA,"07/01/2020, 21:47:16"
1,1,albany,-79.956088,107.500708,81.00,67,50,5.03,42.60,-73.97,US,"07/01/2020, 14:42:47"
2,2,puerto guzman,1.242171,-76.251296,88.74,49,15,3.27,0.97,-76.59,CO,"07/01/2020, 13:47:23"
3,3,atar,20.244804,-13.782911,110.71,7,99,7.81,20.52,-13.05,MR,"07/01/2020, 18:45:22"
4,4,butler,40.932958,-80.106901,90.00,55,20,3.36,39.45,-84.57,US,"07/01/2020, 14:47:24"


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#enable gmaps extension for this notebook
# !pip install gmaps
# jupyter nbextension enable --py gmaps

# Configure gmaps
gmaps.configure(api_key=g_key)

#set locations (lat, long) data for the heatmap
locations = cities_examined.loc[:, ["Actual Latitude", "Actual Longitude"]]


# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_examined["Humidity (%)"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
select_cities = cities_examined[(cities_examined["Cloudiness"]<20) & (cities_examined["Humidity (%)"]<30) &
                                (cities_examined["Max Temperature (F)"]>70) & (cities_examined["Max Temperature (F)"]<80)]
select_cities

,Unnamed: 0,City,Approx. Latitude,Approx. Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Actual Latitude,Actual Longitude,Country,Datetime (Local)
12,12,korla,41.431506,87.504184,78.39,17,7,9.60,41.76,86.15,CN,"07/02/2020, 02:43:59"
311,311,lucapa,-8.109527,18.726333,73.27,29,0,3.53,-8.42,20.74,AO,"07/01/2020, 19:48:30"
331,331,gallup,36.209935,-108.953807,77.00,12,1,14.99,35.53,-108.74,US,"07/01/2020, 12:44:52"
372,372,cochabamba,-16.446932,-66.198950,77.00,9,3,5.50,-17.39,-66.16,BO,"07/01/2020, 14:48:44"
452,452,ogden,41.242218,-111.946871,75.99,22,1,9.17,41.22,-111.97,US,"07/01/2020, 12:49:00"
467,467,gillette,44.367882,-105.609492,75.99,25,1,19.46,44.29,-105.50,US,"07/01/2020, 12:49:03"
482,482,beyneu,46.535817,55.223039,72.45,24,0,9.60,45.32,55.20,KZ,"07/01/2020, 23:49:06"
520,520,shu,44.557238,72.525295,76.71,28,16,10.63,43.60,73.76,KZ,"07/02/2020, 00:49:15"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = select_cities.loc[:, ["City", "Actual Latitude", "Actual Longitude", "Country", "Datetime (Local)"]]

hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

for i, row in hotel_df.iterrows():
    try:
        curr_lat = row["Actual Latitude"]
        curr_lng = row["Actual Longitude"]
        params["location"] = f"{curr_lat},{curr_lng}"

        base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        name_address = requests.get(base_places_url, params)
        
        json_resp = name_address.json()
        
        hotel_name = json_resp["results"][0]["name"]
        
        hotel_df["Hotel Name"][i] = hotel_name
        
    except (KeyError, ValueError, IndexError):
        print("Hotel not found for the city of {}".format(row["City"]))

# print(json_resp["results"][0]["name"])
hotel_df

C:\Users\patri\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Actual Latitude,Actual Longitude,Country,Datetime (Local),Hotel Name
12,korla,41.76,86.15,CN,"07/02/2020, 02:43:59",Gulisitan Hotel
311,lucapa,-8.42,20.74,AO,"07/01/2020, 19:48:30",Residência Ulisses Dos Santos
331,gallup,35.53,-108.74,US,"07/01/2020, 12:44:52",El Capitan Motel
372,cochabamba,-17.39,-66.16,BO,"07/01/2020, 14:48:44",Hotel Aranjuez Cochabamba
452,ogden,41.22,-111.97,US,"07/01/2020, 12:49:00",Hampton Inn & Suites Ogden
467,gillette,44.29,-105.50,US,"07/01/2020, 12:49:03",National 9 Inn - Gillette WY
482,beyneu,45.32,55.20,KZ,"07/01/2020, 23:49:06",NUR
520,shu,43.60,73.76,KZ,"07/02/2020, 00:49:15",Shyngyzkhan


In [15]:
#drop locations where there are no hotels within 5000 meters of the latitude/longitude
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df

,City,Actual Latitude,Actual Longitude,Country,Datetime (Local),Hotel Name
12,korla,41.76,86.15,CN,"07/02/2020, 02:43:59",Gulisitan Hotel
311,lucapa,-8.42,20.74,AO,"07/01/2020, 19:48:30",Residência Ulisses Dos Santos
331,gallup,35.53,-108.74,US,"07/01/2020, 12:44:52",El Capitan Motel
372,cochabamba,-17.39,-66.16,BO,"07/01/2020, 14:48:44",Hotel Aranjuez Cochabamba
452,ogden,41.22,-111.97,US,"07/01/2020, 12:49:00",Hampton Inn & Suites Ogden
467,gillette,44.29,-105.50,US,"07/01/2020, 12:49:03",National 9 Inn - Gillette WY
482,beyneu,45.32,55.20,KZ,"07/01/2020, 23:49:06",NUR
520,shu,43.60,73.76,KZ,"07/02/2020, 00:49:15",Shyngyzkhan


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Actual Latitude", "Actual Longitude"]]

In [17]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))